In [123]:
import numpy as np
import matplotlib.pyplot as plt 
from find_peak_slope import *
from zacLib import awgn
%matplotlib Qt5

In [124]:
# find_peak_slope_from_rd('sig0.rd',400,50,1,0)

In [125]:
def bell(n,h,sigma=1):
    return h*np.exp(-np.arange(-n//2,n//2)**2/2/sigma**2)

def square(n,h,alpha=0,ripple=0):
    if alpha>n//2: alpha = n//2
    s = np.zeros(n)
    s[0:alpha] = np.cos(np.linspace(np.pi,np.pi*2,alpha))+1
    s[n-alpha:n] = np.cos(np.linspace(0,np.pi,alpha))+1
    s[alpha:n-alpha] = 2+ripple*np.sin(np.linspace(0,12*np.pi,n-2*alpha))
    return 0.5*h*s

def stair(n,h1,h2,alpha=0,sigma=1,ripple=0):
    return bell(n,h1,sigma)+square(n,h2,alpha,ripple)


In [126]:
n=int(8000)
x = np.zeros(n)
x[100:1000] += square(900,1,alpha=0,ripple=0)
x[1100:2000] += square(900,1,alpha=100,ripple=0)
x[2100:3000] += square(900,1,alpha=300,ripple=0)
x[3100:4000] += square(900,1,alpha=450,ripple=0)
x[4100:5000] += square(900,0.5,alpha=0,ripple=0)
x[5100:6000] += square(900,0.5,alpha=100,ripple=0)
x[6100:7000] += square(900,0.5,alpha=300,ripple=0)
x[7100:8000] += square(900,0.5,alpha=450,ripple=0)

# stairs
# x[100:1000] = stair(900,1,0.1,alpha=100,sigma=30,ripple=0)
# x[1100:2000] = stair(900,0.4,1,alpha=200,sigma=10,ripple=0)
# x[2100:3000] = stair(900,1,1,alpha=30,sigma=50,ripple=0)
# x[3100:4000] = stair(900,1,1,alpha=300,sigma=30,ripple=0)
nx = awgn(x,None,0.05)
# plt.figure()
# plt.plot(nx,'.',markersize=1)
# plt.plot(x)

# plt.show()


In [127]:
width = 150
slope_th = 20

sp = nx
sp = np.concatenate([np.zeros(width),sp,np.zeros(width)])
sm = mySmooth(sp,width,n=1) # n control shape of smooth kernel (lager n, kernel is more like gaussian(bell) shape)
dsp = dev(sp,width) # calculate derivative/slope
# ploc = np.where(dsp>slope_th)[0] # slope > slope_th ==> rising edge 
# vloc = np.where(dsp<-slope_th)[0] # slope < -slope_th ==> falling edge
# new_peak,new_pits = filter_extremum(dsp,ploc,vloc)
# sm,dsp,new_peak,new_pits,avg_frq,max_frq,\
#         dev_band,half_left,half_right,half_band = find_peak_slope(nx,width,slope_th)
# plot_all(nx,sm,dsp,slope_th,new_peak,new_pits,avg_frq,max_frq,dev_band,half_left,half_right,1)

In [128]:
ploc=[]
vloc=[]
maxPeak = 0
minPeak = 0
current = 'neither' # state machine: 
for i,d in enumerate(dsp):
    if d>slope_th:
        current = 'rising'
        if d>maxPeak:
            maxPeak = d
            pl = i
    elif d<-slope_th:
        current = 'falling'
        if d<minPeak:
            minPeak = d
            vl = i
    else:
        if current == 'rising':
            if len(ploc)-len(vloc)==1:
                ploc[-1] = pl
            else:
                ploc.append(pl)
            maxPeak = 0
        elif current == 'falling':
            if len(ploc)-len(vloc)==1:
                vloc.append(vl)
            minPeak = 0
        current = 'neither'
new_peak=np.array(ploc)
new_pits=np.array(vloc)

In [129]:
plt.figure()
f=1
f_axis = np.arange(sm.size)*f
ax1 = plt.subplot(211)
plt.plot(np.concatenate([np.zeros(width),x,np.zeros(width)]),label='Origin')
plt.plot(sp,'.',markersize=1,label='Raw') # if sp.size>1e6, plot sp as scatter will be very slow
plt.plot(f_axis,sm,label='Smoothed')
plt.plot(new_peak*f,sm[new_peak],'r^',label='rising')
plt.plot(new_pits*f,sm[new_pits],'gv',label='falling')
# plt.plot(half_left*f,sm[half_left],'b>',label='half_left')
# plt.plot(half_right*f,sm[half_right],'b<',label='half_right')
# plt.plot(avg_frq*f,sm[avg_frq],'d',label='avg_peak')
# plt.plot(max_frq*f,sm[max_frq],'s',label='max_peak')
plt.legend(loc='right')
plt.subplot(212,sharex=ax1)
plt.plot(f_axis,dsp)
plt.plot(f_axis,np.ones(sm.size)*slope_th,'--')
plt.plot(f_axis,np.ones(sm.size)*-slope_th,'--')
plt.plot(new_peak*f,dsp[new_peak],'r^',markersize=4)
plt.plot(new_pits*f,dsp[new_pits],'gv',markersize=4)
plt.show()

In [130]:
new_pits

array([1149, 2098, 3004, 3919, 5149, 6101, 6993, 7898])